# Strongest features selection (with Random Forest)
(based on Factor Analysis of Mixed Data/FAMD)

In [24]:
import pandas as pd

In [25]:
random_state = 42

# Load data

In [26]:
train_values = pd.read_csv('../data/train_values.csv', index_col='building_id')
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [27]:
train_labels = pd.read_csv('../data/train_labels.csv', index_col='building_id')
train_labels.head()

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


# Explore features

In [28]:
# Categorical columns = 
categorical_columns = [c for c in train_values.select_dtypes(include=['object'])]
for c in categorical_columns:
    print(c)

land_surface_condition
foundation_type
roof_type
ground_floor_type
other_floor_type
position
plan_configuration
legal_ownership_status


In [29]:
numerical_columns = list(set(train_values.columns) - set(categorical_columns))
numerical_columns

['has_secondary_use_industry',
 'has_superstructure_cement_mortar_brick',
 'has_superstructure_other',
 'has_superstructure_rc_non_engineered',
 'geo_level_3_id',
 'area_percentage',
 'has_secondary_use_gov_office',
 'has_superstructure_mud_mortar_stone',
 'has_secondary_use_use_police',
 'has_secondary_use_school',
 'has_secondary_use_institution',
 'geo_level_2_id',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_cement_mortar_stone',
 'has_superstructure_stone_flag',
 'has_secondary_use_rental',
 'has_superstructure_timber',
 'has_superstructure_bamboo',
 'count_floors_pre_eq',
 'has_secondary_use_hotel',
 'has_superstructure_adobe_mud',
 'count_families',
 'age',
 'geo_level_1_id',
 'has_superstructure_rc_engineered',
 'has_secondary_use',
 'has_secondary_use_agriculture',
 'has_secondary_use_health_post',
 'height_percentage',
 'has_secondary_use_other']

In [30]:
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


### Drop duplicates

In [31]:
duplicate_index_mask = train_values.index.duplicated(keep='first')
train_values = train_values[~duplicate_index_mask]
train_labels = train_labels[~duplicate_index_mask]

### Drop the index column

In [32]:
# Drop building_id (index) from X and y
train_values.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
# Convert y to a Series instead of a DataFrame
train_labels = train_labels['damage_grade']


In [33]:
# convert geo data to categorical
def fixup_geo_data(data):
    geo_cols = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']
    for c in geo_cols:
        data[c] = data[c].astype('object')
fixup_geo_data(train_values)

## Split the data

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_values, train_labels, train_size=0.8, test_size=0.2, random_state=random_state)

All possible values from all categorical columns are present in the train set

In [35]:
# X_train['volume_percentage']=X_train['area_percentage'] * X_train['height_percentage']
# X_valid['volume_percentage']=X_valid['area_percentage'] * X_valid['height_percentage']


## Select only the features contributing mostly to the variance
(based on FAMD/PCA)

Features contibuting to C1
- geo_level_3_id       0.008093
- foundation_type      0.006557
- roof_type            0.006494
- ground_floor_type    0.006341
- other_floor_type     0.005964
- geo_level_2_id       0.005939

Features contibuting to C2
- geo_level_3_id     0.014831
- geo_level_2_id     0.013754
- geo_level_1_id     0.009466
- foundation_type    0.007587

In [36]:
cols_famd = ['geo_level_3_id', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'geo_level_2_id']

## Encode categorical values

In [37]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import TargetEncoder

In [38]:
categorical_transformer = Pipeline(steps=[
        ('target', TargetEncoder(target_type='continuous'))
    ])
preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, cols_famd),
             ("numerical", "passthrough", numerical_columns),
        ])

In [39]:
# the model
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=300, max_depth=None, random_state=random_state)

In [40]:
clf = Pipeline(
       steps=[('preprocessor', preprocessor),
              ('model', random_forest)])

X = X_train.copy()
y = y_train.copy()
clf.fit(X, y)
score = clf.score(X_valid.copy(), y_valid.copy())
print(f"Evaluated model: score: {score}")

Evaluated model: score: 0.7456303601235587


# Prediction on validation data

In [41]:
preds_valid = clf.predict(X_valid)

In [42]:
from sklearn.metrics import f1_score
print(f"Score on validation data: {f1_score(y_valid, preds_valid, average='micro')}")

Score on validation data: 0.7456303601235587


# Prediction on test data

In [43]:
X_test = pd.read_csv('../data/test_values.csv', index_col='building_id')
# Drop building_id (index)
X_test.reset_index(drop=True, inplace=True)
fixup_geo_data(X_test)

In [44]:
preds_test = clf.predict(X_test)

# Results submission

In [45]:
submission_format = pd.read_csv('../data/submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=preds_test,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [46]:
my_submission.to_csv('submission.csv')